## 最終課題

In [3]:
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import time
from collections import deque

# スタートURL（トップページ）
start_url = "https://www.musashino-u.ac.jp/"
domain = urlparse(start_url).netloc

# User-Agent設定（botとみなされないように）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.5993.70 Safari/537.36"
}

# 結果を格納する辞書
site_dict = {}

# BFS（幅優先探索）で全ページを巡回
queue = deque([start_url])
visited = set()

# 除外する拡張子（画像・PDFなど）
exclude_ext = (".jpg", ".jpeg", ".png", ".gif", ".svg", ".pdf", ".zip", ".mp4", ".doc", ".docx")

while queue:
    url = queue.popleft()

    if url in visited:
        continue
    visited.add(url)

    # ファイル系URLは除外
    if url.lower().endswith(exclude_ext):
        continue

    try:
        res = requests.get(url, headers=headers, timeout=10)
        if res.status_code != 200:
            continue
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "html.parser")

        # コメントアウト内のリンクを除去
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # タイトルを取得
        title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
        site_dict[url] = title
        print(f"✔ {url} → {title}")

        # 全<a>タグのリンクを収集
        for a in soup.find_all("a", href=True):
            link = urljoin(url, a["href"])  # 絶対URLに変換
            parsed = urlparse(link)

            # 同一ドメインのみ & 除外拡張子でないリンク
            if parsed.netloc == domain and not link.lower().endswith(exclude_ext):
                if link not in visited:
                    queue.append(link)

        # 負荷対策（アクセス間隔）
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Error fetching {url}: {e}")
        continue

print("\n=== 抽出完了 ===")
print(f"総ページ数：{len(site_dict)}")
print(site_dict)

print(f"探索中URL数: {len(queue)} / 訪問済み: {len(visited)} / 登録済み: {len(site_dict)}")

✔ https://www.musashino-u.ac.jp/ → 武蔵野大学
✔ https://www.musashino-u.ac.jp/#main → 武蔵野大学
✔ https://www.musashino-u.ac.jp/access.html → 交通アクセス | 武蔵野大学
✔ https://www.musashino-u.ac.jp/admission/request.html → 資料請求 | 入試情報 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/contact.html → お問い合わせ | 武蔵野大学
✔ https://www.musashino-u.ac.jp/prospective-students.html → 武蔵野大学で学びたい方 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/students.html → 在学生の方 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/alumni.html → 卒業生の方 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/parents.html → 保護者の方 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/business.html → 企業・研究者の方 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/guide/ → 大学案内 | 武蔵野大学
✔ https://www.musashino-u.ac.jp/guide/profile/ → 大学紹介 | 大学案内 | 武蔵野大学


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import time
from collections import deque

start_url = "https://www.musashino-u.ac.jp/"
base_domain = "musashino-u.ac.jp"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

exclude_ext = (".jpg", ".jpeg", ".png", ".gif", ".svg", ".pdf", ".zip", ".mp4", ".doc", ".docx")

queue = deque([start_url])
visited = set()
site_dict = {}

while queue:
    url = queue.popleft()
    if url in visited:
        continue
    visited.add(url)

    if url.lower().endswith(exclude_ext):
        continue

    try:
        res = requests.get(url, headers=headers, timeout=10)
        if res.status_code != 200:
            continue
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "html.parser")

        # コメント除去
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
        site_dict[url] = title

        # 進行ログ
        print(f"[{len(site_dict)}] {url} → {title}")

        for a in soup.find_all("a", href=True):
            link = urljoin(url, a["href"])
            parsed = urlparse(link)

            # サブドメイン含む musashino-u.ac.jp ドメイン内のみ
            if base_domain in parsed.netloc and not link.lower().endswith(exclude_ext):
                # URL正規化（末尾スラッシュ統一）
                norm_link = link.rstrip('/')
                if norm_link not in visited:
                    queue.append(norm_link)

        time.sleep(0.5)

    except Exception as e:
        print(f"⚠️ {url} : {e}")
        continue

print(f"\n=== 抽出完了 ===\n総ページ数: {len(site_dict)}")

[1] https://www.musashino-u.ac.jp/ → 武蔵野大学
[2] https://www.musashino-u.ac.jp/#main → 武蔵野大学
[3] https://www.musashino-u.ac.jp → 武蔵野大学
[4] https://ef.musashino-u.ac.jp/donation → ご寄付のお願い | 学校法人武蔵野大学
[5] https://www.musashino-u.ac.jp/access.html → 交通アクセス | 武蔵野大学
[6] https://www.musashino-u.ac.jp/admission/request.html → 資料請求 | 入試情報 | 武蔵野大学
[7] https://www.musashino-u.ac.jp/contact.html → お問い合わせ | 武蔵野大学
[8] https://www.musashino-u.ac.jp/prospective-students.html → 武蔵野大学で学びたい方 | 武蔵野大学
[9] https://www.musashino-u.ac.jp/students.html → 在学生の方 | 武蔵野大学
[10] https://www.musashino-u.ac.jp/alumni.html → 卒業生の方 | 武蔵野大学
[11] https://www.musashino-u.ac.jp/parents.html → 保護者の方 | 武蔵野大学
[12] https://www.musashino-u.ac.jp/business.html → 企業・研究者の方 | 武蔵野大学
[13] https://www.musashino-u.ac.jp/guide → 大学案内 | 武蔵野大学
[14] https://www.musashino-u.ac.jp/guide/profile → 大学紹介 | 大学案内 | 武蔵野大学
[15] https://www.musashino-u.ac.jp/guide/activities → 大学の取り組み | 大学案内 | 武蔵野大学
[16] https://www.musashino-u.ac.jp/guide/campus → キャ

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import csv
import time

start_url = "https://www.musashino-u.ac.jp/"
base_domain = "musashino-u.ac.jp"
exclude_ext = (".jpg", ".jpeg", ".png", ".gif", ".pdf", ".zip", ".doc", ".docx", ".xls", ".xlsx", ".ppt", ".pptx", ".mp4", ".mp3", ".svg")
output_file = "musashino_univ_sitemap.csv"

# User-Agent（アクセスを弾かれないように設定）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.5993.118 Safari/537.36"
}

visited = set()
pages = {}
queue = deque([start_url])

while queue:
    url = queue.popleft()
    if url in visited:
        continue
    visited.add(url)

    try:
        res = requests.get(url, headers=headers, timeout=10)
        if res.status_code != 200 or "text/html" not in res.headers.get("Content-Type", ""):
            continue

        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "html.parser")
        title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
        pages[url] = title

        # ページ内リンクを探索
        for link in soup.find_all("a", href=True):
            href = link["href"].strip()
            if href.startswith("#") or href.startswith("mailto:") or href.startswith("javascript:"):
                continue

            new_url = urljoin(url, href)
            parsed = urlparse(new_url)

            # サブドメイン含めた musashino-u.ac.jp 内のみ対象
            if base_domain in parsed.netloc and not new_url.lower().endswith(exclude_ext):
                if new_url not in visited:
                    queue.append(new_url)

        # 負荷軽減（ランダムスリープ）
        time.sleep(random.uniform(0.5, 1.5))

    except Exception as e:
        print(f"Error at {url}: {e}")

with open(output_file, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["URL", "Title"])
    for url, title in pages.items():
        writer.writerow([url, title])

print(f"\n✅ 完了！取得ページ数：{len(pages)}件")
print(f"💾 出力ファイル：{output_file}")        